<a href="https://colab.research.google.com/github/AfaqSaeed/Extract_Frames/blob/master/Generate_Auto_Labeled_Sets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# from google.colab.patches import cv2_imshow
# drive.mount('/content/gdrive')
 
# proj_dir = "/content/gdrive/MyDrive/deeplabv3/pytorch/"
# import shutil
# shutil.copy(proj_dir+'datasets.py','/content/')
# shutil.copy(proj_dir+'/utils/'+'utils.py','/content/')
# shutil.copy(proj_dir+'/model/'+'resnet.py','/content/')
# shutil.copy(proj_dir+'/model/'+'aspp.py','/content/')
# shutil.copy(proj_dir+'/model/'+'deeplabv3_old.py','/content/')

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

from google.colab.patches import cv2_imshow # Debugging Import
import sys
import pandas
import torchvision
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import matplotlib.pyplot as plt
import time
import numpy as np
import cv2
import os
from google.colab import drive
import gc
import glob
 
print('Required Libraries Imported')

Mounted at /content/gdrive
Required Libraries Imported


In [4]:
%cd "/content/gdrive/MyDrive/GoPro data/DL Labelling Data/"
import os
import cv2
import pandas as pd
from tqdm import tqdm


def get_secs(text):
    array = text.split(",",-1)
    result =[]
    for i in range(len(array)):
        start, stop  = array[i].split("-")
        start = int(float(start.strip()))*60 +(float(start.strip())%1)*100
        stop = int(float(stop.strip()))*60 +(float(stop.strip())%1)*100
        result.append((start,stop))
    return result
videodir = "ExtractFrames/"
filelist = os.listdir(videodir)
videos = [i for i in filelist if i.lower().find(".mp4") !=-1]
print(videos)

sheet = pd.read_excel("DefectMarking.xlsx")

def endptstoframes(start,stop,video,folder):
    print(video)
    cap = cv2.VideoCapture(videodir + video.upper())
    fps = cap.get(cv2.CAP_PROP_FPS)
    print("FPS: ",fps)
    length = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    print("Length: ",length)
    
    startframe = int(start*fps)
    stopframe = int(stop*fps)
    difference = int(stopframe-startframe)
    print(startframe,stopframe)
    cap.set(cv2.CAP_PROP_POS_FRAMES,startframe)
    for i in range(0,difference,7):
        
        ret,frame=cap.read()
        
        if frame is not None:
            print(f"Frame :{int(i+startframe)}")
            print(f"Written to {folder}/{video[:-4]}_Frame_{int(i+startframe)}.png")
            cv2.imwrite(f"{folder}/{video[:-4]}_Frame_{int(i+startframe)}.png",frame)

for vid in tqdm(videos):
    for i in range(0,len(sheet.index)): 
        if sheet['Video'][i] == vid[:-4]:
            print("Video :", vid)
            for feild in sheet.keys()[2:]:
                         
                
                string = sheet[feild].astype("string")[i]
                print(feild, string)
                if type(string)!=pd._libs.missing.NAType:
                    intervals = get_secs(string)
                    
                    for start,stop in intervals:
                        endptstoframes(start,stop,vid,feild)
    



/content/gdrive/MyDrive/GoPro data/DL Labelling Data
[]


0it [00:00, ?it/s]


In [5]:
!git clone https://github.com/StradaImaging/Cyclepath-Inspection-Pipeline.git 
%cd Cyclepath-Inspection-Pipeline
!pip install visdom
from utils import utils
import network
from utils import ext_transforms as et

# from metrics import StreamSegMetrics

proj_dir_2      = "/models/"
model_id_2      = "deeplabv3plus_resnet101_lr=0.02_decay=0.0001"
model_2_weights = "/content/gdrive/MyDrive/DeepLabV3Plus/logs/deeplabv3plus_resnet101_lr=0.02_decay=0.0001/checkpoints/deeplabv3plus_resnet101_lr=0.02_decay=0.0001_best_epoch_114.tar"
state_dict_2    = torch.load(model_2_weights,map_location="cpu")

print(model_2_weights)
print(state_dict_2['epoch'])

network_2 = network.modeling.__dict__["deeplabv3plus_resnet101"](num_classes=10, output_stride=16)
network.convert_to_separable_conv(network_2.classifier)

import torch
network_2.load_state_dict(state_dict_2["model_state_dict"])

Cloning into 'Cyclepath-Inspection-Pipeline'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 163 (delta 63), reused 101 (delta 24), pack-reused 0
Receiving objects: 100% (163/163), 338.47 KiB | 8.46 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/gdrive/MyDrive/GoPro data/DL Labelling Data/Cyclepath-Inspection-Pipeline
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 676 kB 10.6 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=9d27da4383378074870b79553899bbb3ede15d01969531968953a47d26aec21e
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

<All keys matched successfully>

In [6]:
# import sys
# import time
# import pandas
# import torchvision
# import torch
# import torch.utils.data
# import torch.nn as nn
# from torch.autograd import Variable
# import torch.optim as optim
# import torch.nn.functional as F
# import torchvision.models as models
# import torchvision.transforms as transforms
# from torch.utils.tensorboard import SummaryWriter
# import numpy as np
# import pickle
# import matplotlib
# matplotlib.use("Agg")
# import matplotlib.pyplot as plt
# import cv2
# import os
# from google.colab import drive
# import gc
 
# os.sys.path.append('/content/gdrive/My Drive/deeplabv3/pytorch')
# os.sys.path.append('/content/gdrive/My Drive/deeplabv3/pytorch/model')
# os.sys.path.append('/content/gdrive/My Drive/deeplabv3/pytorch/utils')
 
# import resnet
# from resnet import ResNet18_OS16, ResNet34_OS16, ResNet50_OS16, ResNet101_OS16, ResNet152_OS16, ResNet18_OS8, ResNet34_OS8
# from datasets import DatasetTrain, DatasetVal
# from utils import label_img_to_color, add_weight_decay, MyModel
# from deeplabv3_old import DeepLabV3

# !pip install tensorboardX
# from tensorboardX import SummaryWriter
# from tqdm.notebook import tqdm
 
# !pip install tensorboard
# %load_ext tensorboard

# import warnings
# warnings.filterwarnings('ignore')
# warnings.simplefilter('ignore')

print('Required Libraries Imported')

# # We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
# !tar -xzvf /content/gdrive/My\ Drive/yolov3/darknet/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
# !chmod a+r /usr/local/cuda/include/cudnn.h

# # Now we check the version we already installed. Can comment this line on future runs
# !cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

def label_img_to_color_10(img):
    img=img.astype(np.uint8)
    lut=np.ones((256,1,3),dtype=np.uint8)
    
    ine=np.array([[[128,123,123],#background 0
                   [48,131,221],#signboard blue 1 
                   [0,255,0],#siderail green  2
                   [128,146,22],#cars red 3
                   [49,147,245],#traffic cone orange 4
                   [255,255,0],#markerposts cyan 5
                   [12,89,11],#lane markings magenta 6
                   [112,32,33],#road yellow 7
                   [122,36, 122],#sidewalk white 8
                   [41, 41, 140],#pole brown 9 
                   ]])
    
    lut[0:10,:,:]=ine.reshape(10,1,3)
    img_color=cv2.applyColorMap(img,lut)

    return img_color

Required Libraries Imported


# Predict on Video

In [ ]:
def predictvideo(weights_path, video_dir_path, save_path, num_clasees,cuda=True):
  proj_dir="/content/gdrive/MyDrive/PavementDefects/deeplabv3_pytorch/"
  model_id = "potcrack"
  network   = network_2
  #print(network)
  #print(new_weights_path)
  if cuda:            
    device = torch.device("cuda:0")
    network.cuda()
    network.to(device)
    Tensor = torch.cuda.FloatTensor 
    LongTensor = torch.cuda.LongTensor 
    # state_dict=torch.load(weights_path,map_location="cuda:0")
  else:
    device = torch.device("cpu")
    network.cuda()
    network.to(device)
    Tensor=torch.FloatTensor
    LongTensor=torch.LongTensor
    # state_dict=torch.load(weights_path,map_location="cpu")
  # network.load_state_dict(state_dict)
  height, width, layers =896, 448, 3
  size = (width,height)
    
  dir=video_dir_path
  if not dir[-1]=="/":
    dir=dir+"/"

  for vid_path in os.listdir(dir):
          print(vid_path)
        #  with torch.autograd.profiler.profile(use_cuda=True) as prof:  
          with torch.no_grad(): # (corresponds to setting volatile=True in all variables, this is done during inference to reduce memory consumption)

            if not os.path.exists(save_path):
              os.mkdir(save_path)


            path = dir + vid_path
            print("PATH" + path)
            print("SAVE DIR " + save_path + vid_path[:-4] + "_inferred" + vid_path[-4:])
            # REMOVE THIS PART LATER start
            path = dir + vid_path
            # REMOVE THIS PART LATER end

            
            out = cv2.VideoWriter(save_path + vid_path[:-4] + "_inferred" + vid_path[-4:], cv2.VideoWriter_fourcc(*'MPEG'), 1, size)
            cap = cv2.VideoCapture(path)
          #  cap = cv2.VideoCapture(dir+vid_path)
            if (cap.isOpened() == False):
                print("Error opening video stream or file")

    

            # Read until video is completed
            i=0
            
            while(cap.isOpened()):
              with torch.autograd.profiler.profile(use_cuda=True) as prof:  
          
              # Capture frame-by-frame

                ret, frame = cap.read()

                if ret == True:
                  print("Frame "+str(i))
                  i=i+1  
                  img = frame # (shape: (1024, 2048, 3))
            #      img = cv2.imread(imgPath)
                  oimg = cv2.resize(img, (448, 448), interpolation=cv2.INTER_NEAREST)
                  img = oimg/255.0
                  img = img - np.array([0.485, 0.456, 0.406])
                  img = img/np.array([0.229, 0.224, 0.225]) # (shape: (512, 1024, 3))
                  img = np.transpose(img, (2, 0, 1)) # (shape: (3, 512, 1024))
                  img = img.astype(np.float32)
                  img = torch.from_numpy(img)
                  img_batch = torch.stack((img,img),dim=0)
                  img_batch = Variable(img_batch).cuda()
                    # compute the loss:
                  outputs = network(img_batch) # (shape: (batch_size, num_classes, img_h, img_w))
                    # Perform inference in the images 
                      
                  _, predicted = torch.max(outputs.data, 1)
                  outputs = outputs.data.cpu().numpy() # (shape: (batch_size, num_classes, img_h, img_w))
                    
                  pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, img_h, img_w))
                  pred_label_imgs = pred_label_imgs.astype(np.uint8)          
                  pred_label_img = pred_label_imgs[0] # (shape: (img_h, img_w))
                  img =oimg  # (shape: (3, img_h, img_w))
                  pred_label_img_color = label_img_to_color_10(pred_label_img)

                  print(pred_label_img_color.shape)
                  savePath = "/content/gdrive/MyDrive/PavementDefects/InferredImages/"
                  cv2.imwrite(savePath,  pred_label_img_color)

                  overlayed_img = cv2.vconcat([img, pred_label_img_color])
                  overlayed_img = overlayed_img.astype(np.uint8)
                  
                  out.write(overlayed_img.astype(np.uint8))
                else:
                 break
              # print(prof)
            out.release()            
            cap.release()

In [ ]:
proj_dir="/content/gdrive/My Drive/DeepROW"
new_weights_path = proj_dir + "/model_utils/weights/updatedClassModel.pth" 
print(new_weights_path)
video_dir_path = "/content/gdrive/MyDrive/PavementDefects/ToInfer"
save_path = "/content/gdrive/MyDrive/PavementDefects/Inferred/"
img_dir_path = "/content/gdrive/MyDrive/25images/Images/"
num_classes=10
predictvideo(new_weights_path, video_dir_path, save_path, num_classes)

/content/gdrive/My Drive/DeepROW/model_utils/weights/updatedClassModel.pth
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\afaqs\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\afaqs\AppData\Local\Temp\ipykernel_19224\1305348752.py", line 8, in <cell line: 8>
    predictvideo(new_weights_path, video_dir_path, save_path, num_classes)
NameError: name 'predictvideo' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\afaqs\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\afaqs\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\afaqs\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", l

# Make Video of Images

In [ ]:
import cv2
import glob

#path = '/content/drive/MyDrive/images2/*.jpg'
path = '/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/*.jpg'
save_path = '/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/'

output_name = 'GX010029_inferred_labels.AVI'

fps = 1
img_array = []
for filename in glob.glob(path):
    print(filename)
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

print(save_path + output_name)
out = cv2.VideoWriter(save_path + output_name,cv2.VideoWriter_fourcc(*'MPEG'), fps, size)
print(len(img_array))

for i in range(len(img_array)):
#    print(img_array[i])
    out.write(img_array[i])
#    cv2_imshow(img_array[i])
out.release()

print('done')

/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/GX010029_mask_frame_15.jpg
/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/GX010029_mask_frame_54.jpg
/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/GX010029_mask_frame_69.jpg
/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/GX010029_mask_frame_84.jpg
/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/GX010029_mask_frame_99.jpg
/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/GX010029_mask_frame_114.jpg
/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/GX010029_mask_frame_129.jpg
/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/GX010029_mask_frame_144.jpg
/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/GX010029_mask_frame_159.jpg
/content/gdrive/MyDrive/PavementDefects/GoProFramesDeepLabV3Output/GX010029_mask_frame_174.jpg
/content/gdrive/MyDrive/PavementDefects/GoProFramesDeep

In [ ]:
from AnalyzeCyclePath import AnalyzeCyclePath

proj_dir_2      = "./models/"
model_id_2      = "deeplabv3plus_resnet101_lr=0.02_decay=0.0001"
model_2_weights = "/content/gdrive/MyDrive/DeepLabV3Plus/logs/deeplabv3plus_resnet101_lr=0.02_decay=0.0001/checkpoints/deeplabv3plus_resnet101_lr=0.02_decay=0.0001_best_epoch_114.tar"
analyze = AnalyzeCyclePath(model_2_weights,"./mlp-8x8.joblib")


/content/gdrive/MyDrive/DeepLabV3Plus/logs/deeplabv3plus_resnet101_lr=0.02_decay=0.0001/checkpoints/deeplabv3plus_resnet101_lr=0.02_decay=0.0001_best_epoch_114.tar
deeplabv3 loaded
./mlp-8x8.joblib
sklearnmodel loaded


# Predict on Images

In [ ]:
def predictImages(weights_path, num_clasees, imgs_paths, save_paths, cuda=True):
  network   = network_2
  #print(network)
  #print(new_weights_path)
  if cuda:            
    device = torch.device("cuda:0")
    network.cuda()
    network.to(device)
    Tensor = torch.cuda.FloatTensor 
    LongTensor = torch.cuda.LongTensor 
    # state_dict=torch.load(weights_path,map_location="cuda:0")
  else:
    device = torch.device("cpu")
    # network.cuda()
    network.to(device)
    Tensor=torch.FloatTensor
    LongTensor=torch.LongTensor
    # state_dict=torch.load(weights_path,map_location="cpu")
  # network.load_state_dict(state_dict)
  height, width, layers =896, 448, 3
  size = (width,height)
  
  Crackd = False
  Potholed = False
  Paintd = True


  for img_path, save_path in zip(imgs_paths, save_paths):
          print(img_path, save_path)
        #  with torch.autograd.profiler.profile(use_cuda=True) as prof:  
          with torch.no_grad(): # (corresponds to setting volatile=True in all variables, this is done during inference to reduce memory consumption)
              
              img = cv2.imread(img_path)
              h,w = 1080,1920
              oimg = cv2.resize(img, (1280, 960), interpolation=cv2.INTER_NEAREST)
              img = oimg/255.0
              img = img - np.array([0.485, 0.456, 0.406])
              img = img/np.array([0.229, 0.224, 0.225]) # (shape: (512, 1024, 3))
              img = np.transpose(img, (2, 0, 1)) # (shape: (3, 512, 1024))
              img = img.astype(np.float32)
              img = torch.from_numpy(img)
              img_batch = torch.stack((img,img),dim=0)
              img_batch = Variable(img_batch).cuda()
                # compute the loss:
              outputs = network(img_batch) # (shape: (batch_size, num_classes, img_h, img_w))
                # Perform inference in the images 
                  
              _, predicted = torch.max(outputs.data, 1)
              outputs = outputs.data.cpu().numpy() # (shape: (batch_size, num_classes, img_h, img_w))
                
              pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, img_h, img_w))
              pred_label_imgs = pred_label_imgs.astype(np.uint8)          
              pred_label_img = pred_label_imgs[0] # (shape: (img_h, img_w))
              print(pred_label_img.shape)
              # cv2.imwrite(save_path,  pred_label_img)

              img =oimg  # (shape: (3, img_h, img_w))
              # pred_label_img_color = label_img_to_color_10(pred_label_img)
              

              # print(pred_label_img_color.shape)
              droad = pred_label_img == 7
              # kernel = np.ones((7, 7), np.uint8)

              # droad = cv2.morphologyEx(droad.astype(np.uint8), cv2.MORPH_CLOSE, kernel,iterations=3)

              paint= pred_label_img == 6
              contours, hierarchy = cv2.findContours(droad.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
              road = np.zeros(droad.shape,dtype = bool)
              hi,wi = droad.shape
              for cnt in contours:
                
                len = cv2.arcLength(cnt,True)
                area = cv2.contourArea(cnt)
                
                # print("Area",area)
                if area < 0.1*hi*wi:
                  # print("smol",area)
                  
                  continue
                print("big",area)
                # epsilon = 0.1*len
                # cnt = cv2.approxPolyDP(cnt,epsilon,True)
                hull = cv2.convexHull(cnt)
                road = cv2.fillConvexPoly(road.astype(np.uint8),hull,1)

              
              # print(w,h)
              # print ("Road",road.dtype)
              # print ("Paint",paint.shape)    
              # print ("Pothole",pothole.shape)
              # print ("Crack",crack.shape) 

              print(w,h)
              road = cv2.resize(road.astype(np.uint8),(w,h))
              
              # cv2.imwrite(save_path ,(road*255))

              detection  = np.zeros((h,w),dtype = np.uint8)
              detection = np.where(road,7,detection)
              if Paintd:
                  paint = cv2.resize(paint.astype(np.uint8),(w,h))
                  detection = np.where(paint,5,detection)
 
              if Potholed:
                  pothole = analyze.PotholeDetect(img)
                  pothole = cv2.resize(pothole,(w,h))
                  detection = np.where(pothole,1,detection)
              if Crackd:
                  crack = analyze.CrackDetect(img)
                  crack = cv2.resize(crack,(w,h))
                  detection = np.where(crack,2,detection)

              
     
              label_to_color = np.array([
                      [0,0,0],      #  Background    : Black
                      [0,255,255],  #  Pothole       : Yellow 
                      [0,0,255],    #  Cracks        : Red
                      [255,255,0],  #  Patch         : Cyan
                      [255,0,0],    #  Puddle        : Blue
                      [255,0,255],  #  Paint         : Magenta = Blue + Red
                      [0,255,0],    #  Junk/Debris   : Green
                      [255,255,255],#  Pavement      : White
                  ])
              ColorImage = label_to_color[detection]
              # cv2_imshow(pred_label_img_color)
              cv2.imwrite(save_path,  ColorImage)
              

In [ ]:
  
def load_images_from_folder(folder, save_folder_path):
    imgs_path = []
    save_paths = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            # print(os.path.join(folder,filename), os.path.join(save_folder_path, filename))
            imgs_path.append(os.path.join(folder, filename))
            save_paths.append(os.path.join(save_folder_path, filename))

    return imgs_path, save_paths
#model_id = "potcrack"
#proj_dir="/content/gdrive/MyDrive/PavementDefects/deeplabv3_pytorch/"
#new_weights_path = proj_dir + "training_logs/model_" + model_id + "/checkpoints/" + os.listdir(proj_dir+"training_logs/model_"+model_id+"/checkpoints")[-1] 
#print(new_weights_path)

proj_dir="/content/gdrive/My Drive/DeepROW"
new_weights_path = proj_dir + "/model_utils/weights/updatedClassModel.pth" 

save_path = "/content/gdrive/MyDrive/PavementDefects/Inferred/"
num_classes=10


folder_path = "/content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/"
save_folder_path = "/content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/output/"


if not os.path.exists(save_folder_path):
    os.mkdir(save_folder_path)

imgs_paths, save_paths = load_images_from_folder(folder_path, save_folder_path)


predictImages(new_weights_path, num_classes, imgs_paths, save_paths)

/content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/GX010085_Frame_95.png /content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/output/GX010085_Frame_95.png
(960, 1280)
big 358972.5
1920 1080
/content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/GX010085_Frame_102.png /content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/output/GX010085_Frame_102.png
(960, 1280)
big 394000.5
1920 1080
/content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/GX010085_Frame_109.png /content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/output/GX010085_Frame_109.png
(960, 1280)
big 384238.5
1920 1080
/content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/GX010085_Frame_116.png /content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/output/GX010085_Frame_116.png
(960, 1280)
big 430905.0
1920 1080
/content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/GX010085_Frame_123.png /content/gdrive/MyDrive/GoPro data/DL Labelling Data/Paint/output/GX010085_Frame_123.png
(960, 1

In [9]:
%cd "/content/gdrive/MyDrive/GoPro data/DL Labelling Data/"

import os 
import random
import shutil
save_folder_path = "Haleema's Labelling Task/Task2/"

root = ""
Potholes = root + "Potholes"
Paints = root +"Paint"
StrangePavementArea = root + "Strange Pavement Area"
Heavy = root +"Heavily Cracked Pavement"
LowCracked = root + "Low Cracked Pavement"
Normal = root + "Normal Pavement"

Pothole =["Potholes/"+i for i in os.listdir(Potholes)if not os.path.isdir("Potholes/"+i)]
Paint =["Paint/"+i for i in os.listdir(Paints)if not os.path.isdir("Paint/"+i)]
StrngPav =["Strange Pavement Area/"+i for i in os.listdir(StrangePavementArea) if not os.path.isdir("Strange Pavement Area/"+i)]
heavy =["Heavily Cracked Pavement/"+i for i in os.listdir(Heavy)if not os.path.isdir("Heavily Cracked Pavement/"+i)]
LowCrack =["Low Cracked Pavement/"+i for i in os.listdir(LowCracked)if not os.path.isdir("Low Cracked Pavement/"+i)]
Normalewise = ["Normal Pavement/"+i for i in os.listdir(Normal)if not os.path.isdir("Normal Pavement/"+i)]
fourty = Pothole + Paint + StrngPav + heavy 
print("Total :",int(2.5*len(fourty)))
print("Imp classes quantity :",len(fourty))

print("Proportion form other classes : ",int(len(fourty)*0.75))


x = input("Total Images to give for labelling : ")
implabels = random.sample(fourty,k=int(int(x) * 0.4))
lowcrack = random.sample(LowCrack,k = int(int(x) *0.3)) 
normal = random.sample(Normalewise,k = int(int(x)*0.3))
if not os.path.exists(save_folder_path):
    os.mkdir(save_folder_path)
    os.mkdir(save_folder_path+"/output/")
if not os.path.exists(save_folder_path+"/output/"):
    os.mkdir(save_folder_path+"output/")
complete = implabels + lowcrack + normal
print(complete)
def copyfiles(implabels):
    file = open("Assigned.txt","r+")
    f = file.read()
      
    for img_path in implabels:
        if img_path not in f: 
            img_name = img_path.split("/")[1]
            category = img_path.split("/")[0]
            shutil.copy(img_path,save_folder_path + img_name)
            shutil.copy(category+"/output/"+img_name,save_folder_path +"output/"+ img_name)
            
            shutil.move(img_path,category+"/Labelled/"+img_name)
            shutil.move(category+"/output/"+img_name,category+"/Labelled/output/"+img_name)

            print(img_path,save_folder_path + img_name)
            print(category+"/output/"+img_name,save_folder_path +"output/"+ img_name)
            print(img_path,category+"/Labelled/"+img_name)
            print(category+"/output/"+img_name,category+"/Labelled/output/"+img_name)
            
            # file.write(img_path+"\n")

copyfiles(implabels=complete)



  

/content/gdrive/MyDrive/GoPro data/DL Labelling Data
Total : 417
Imp classes quantity : 167
Proportion form other classes :  125
Total Images to give for labelling : 75
['Heavily Cracked Pavement/GX010093_Frame_1984.png', 'Heavily Cracked Pavement/GX010092_Frame_3731.png', 'Heavily Cracked Pavement/GX010095_Frame_516.png', 'Heavily Cracked Pavement/GX010085_Frame_1759.png', 'Paint/GX010085_Frame_249.png', 'Strange Pavement Area/GX010085_Frame_42.png', 'Heavily Cracked Pavement/GX010097_Frame_905.png', 'Heavily Cracked Pavement/GX010085_Frame_1829.png', 'Heavily Cracked Pavement/GX010089_Frame_921.png', 'Heavily Cracked Pavement/GX010093_Frame_2005.png', 'Heavily Cracked Pavement/GX010085_Frame_1738.png', 'Potholes/GX010097_Frame_342.png', 'Heavily Cracked Pavement/GX010093_Frame_1627.png', 'Heavily Cracked Pavement/GX010092_Frame_173.png', 'Heavily Cracked Pavement/GX010089_Frame_956.png', 'Heavily Cracked Pavement/GX010092_Frame_376.png', 'Paint/GX010085_Frame_284.png', 'Heavily Crack

##Verify Lablled Folders

In [ ]:
%cd "/content/gdrive/MyDrive/GoPro data/DL Labelling Data/"
import os
import shutil
os.listdir("Zaid's Labelling Task")  
zaid = [i for i in os.listdir("Zaid's Labelling Task") if not os.path.isdir("Zaid's Labelling Task/"+i)]  
taimoor = [i for i in os.listdir("Taimoor's Labelling Task") if not os.path.isdir("Taimoor's Labelling Task/"+i)]  
# direct = os.listdir(".")
# print(direct)
label_folders =["Zaid's Labelling Task2"]#"Taimoor's Labelling Task", "Zaid's Labelling Task","Marium's Labelling Task","Haleema's Labelling Task","Adil's Labelling Task",] 
checkfolders = ['Low Cracked Pavement', 'Normal Pavement', 'Heavily Cracked Pavement','Strange Pavement Area',"Potholes","Paint"]
labelled = [(j,i) for j in label_folders for i in os.listdir(j) if not os.path.isdir(j+"/"+i)]
a=0
for j in checkfolders :
    for (f,path) in labelled:
        if os.path.exists(j+"/Labelled/output/"+path):
            a=a+1
            print(j+"/Labelled/output/"+path,f+"/output/"+path)
            shutil.copy(j+"/Labelled/output/"+path,f+"/output/"+path)
          # shutil.move(j+ "/output/"+path,j+"/Labelled/output/")
          # print(j+"/Labelled/"+path)

# # print (checkoutputfolder)
# checkpaths = [j+i for j in checkfolders for i in os.listdir(j) if not os.path.isdir(j+i)]  
# # print(len(checkpaths))
# checkpaths[0]
print(len(labelled),a)


/content/gdrive/MyDrive/GoPro data/DL Labelling Data
Low Cracked Pavement/Labelled/output/GX010097_Frame_1359.png Zaid's Labelling Task2/output/GX010097_Frame_1359.png
Low Cracked Pavement/Labelled/output/GX010096_Frame_579.png Zaid's Labelling Task2/output/GX010096_Frame_579.png
Low Cracked Pavement/Labelled/output/GX010097_Frame_505.png Zaid's Labelling Task2/output/GX010097_Frame_505.png
Low Cracked Pavement/Labelled/output/GX010085_Frame_2549.png Zaid's Labelling Task2/output/GX010085_Frame_2549.png
Low Cracked Pavement/Labelled/output/GX010085_Frame_730.png Zaid's Labelling Task2/output/GX010085_Frame_730.png
Low Cracked Pavement/Labelled/output/GX010093_Frame_3233.png Zaid's Labelling Task2/output/GX010093_Frame_3233.png
Low Cracked Pavement/Labelled/output/GX010085_Frame_1520.png Zaid's Labelling Task2/output/GX010085_Frame_1520.png
Low Cracked Pavement/Labelled/output/GX010085_Frame_779.png Zaid's Labelling Task2/output/GX010085_Frame_779.png
Low Cracked Pavement/Labelled/outpu